In [ ]:
pip install -r requirements.txt

In [ ]:
# Read the mm23 file from the Datasets folder which consists of CPIH data
import pandas as pd
path = '../../Datasets/uk-cost-of-living-dashboard/mm23.csv'
cpi_data = pd.read_csv(path)
cpi_data.head()

In [ ]:
# Read the iphrp file from the Datasets folder which consists of IPHRP data
path = '../../Datasets/uk-cost-of-living-dashboard/iphrp.csv'
iphrp_data = pd.read_csv(path, skiprows=2)
iphrp_data.head()

In [ ]:
# The file contains unwanted metadata and columns. The goal is to filter the required columns
keep_columns = [
    'Title',
    'CPIH INDEX 00: ALL ITEMS 2015=100',
    'CPIH ANNUAL RATE 00: ALL ITEMS 2015=100',
    'CPIH INDEX 04: Housing, water, electricity, gas and other fuels 2015=100',
    'CPIH ANNUAL RATE 04 : HOUSING, WATER AND FUELS 2015=100',
    'CPIH INDEX 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100',
    'CPIH ANNUAL RATE 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100'
]

cpi_data = cpi_data[keep_columns]
cpi_data.head()

In [ ]:
# Rename the columns so they are easy to read and perform operations
cpi_data.rename(columns={
    'Title': 'Date',
    'CPIH INDEX 00: ALL ITEMS 2015=100': 'CPIH_Index_All',
    'CPIH ANNUAL RATE 00: ALL ITEMS 2015=100': 'CPIH_Inflation_All',
    'CPIH INDEX 04: Housing, water, electricity, gas and other fuels 2015=100': 'CPIH_Index_Housing',
    'CPIH ANNUAL RATE 04 : HOUSING, WATER AND FUELS 2015=100': 'CPIH_Inflation_Housing',
    'CPIH INDEX 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100': 'CPIH_Index_Food',
    'CPIH ANNUAL RATE 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100': 'CPIH_Inflation_Food'
}, inplace=True)

cpi_data.head()


In [ ]:
# The Data consists of unwanted rows. Filter the rows to find the rows with year and month
import re

monthly_df = cpi_data[cpi_data['Date'].str.match(r'^\d{4}\s[A-Z]{3}$', na=False)]

monthly_df.head()

In [ ]:
# add a new column by Formatting the date so you can perform operations 
monthly_df['Formatted_Date'] = pd.to_datetime(monthly_df['Date'], format='%Y %b')
monthly_df.head()

In [ ]:
# Filter the data to last 25 years and reset the index 
final_df = monthly_df[monthly_df['Formatted_Date'] >= pd.to_datetime("2001-01-01")]
cpi_df = final_df.reset_index(drop=True)
cpi_df.head()

In [ ]:
# cpi_df = final_df.drop(columns=['Date_dt'], axis=1)
# cpi_df

In [ ]:
iphrp_data.head()

In [ ]:
# Drop all the rows with Year-on-year change. We will focus only on Index
iphrp_index = iphrp_data[iphrp_data['IndexAndYearChange'] == 'Index'].copy()
iphrp_index.head()

In [ ]:
# Drop the Data Marking columns as they are not required
required_col = [col for col in iphrp_index.columns if 'Data Marking' not in str(col)]
required_col_df = iphrp_index[required_col]
required_col_df.head()

In [ ]:
# use the melt function in pandas to bring all the dates into a column and drop the unwanted columns
melted_df = pd.melt(required_col_df, id_vars=['Geography', 'Geography code', 'IndexAndYearChange'],
                    var_name = 'Date',
                    value_name='Rental_Index')
melted_df.drop(columns=['Geography code', 'IndexAndYearChange'], axis=1, inplace=True)
melted_df.head()

In [ ]:
# Drop the null values and format the date to perform operations, sort the date 
melted_df.dropna(subset=['Rental_Index'], inplace=True)
melted_df['Formatted_Date'] = pd.to_datetime(melted_df['Date'], format='%b-%y')
melted_df.sort_values('Formatted_Date', inplace=True)
iphrp_df = melted_df
iphrp_df.head()

In [ ]:
# Print the min and max dates
print(cpi_df['Formatted_Date'].min())
print(cpi_df['Formatted_Date'].max())
print(iphrp_df['Formatted_Date'].min())
print(iphrp_df['Formatted_Date'].max())

In [ ]:
# Filter the CPIH date to match with the IPHRP date

cpih_df = cpi_df[(cpi_df['Formatted_Date'] >= '2005-01-01') & (cpi_df['Formatted_Date'] <= '2023-12-01')]
cpih_df = cpih_df.reset_index(drop=True) 
cpih_df.head()

In [ ]:
cpih_df.head()

In [ ]:
iphrp_df.head()

In [ ]:
cpih_df.dtypes

In [ ]:
iphrp_df.dtypes

In [ ]:
merged_df = pd.merge(cpih_df, iphrp_df, on='Formatted_Date', how='inner')
merged_df.head()

In [ ]:
merged_df.drop(columns=['Date_y', 'Date_x'], inplace=True)
merged_df.rename(columns={
    'Formatted_Date': 'Date'
}, inplace=True)
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
import matplotlib.pyplot as plt

england_df = merged_df[merged_df['Geography'] == 'England']

england_df.plot(x='Date', y='CPIH_Index_All', kind='line', title='England CPIH Index Trend (2005-2023)', grid=True)

plt.show()

In [ ]:
merged_df.plot(x='Date', y='CPIH_Inflation_All', kind='line', title='UK Inflation Trend (2005-2023)', grid=True)

plt.show()

In [ ]:
merged_df['Geography'].unique()

In [ ]:

england_df.plot(x='Date', y='Rental_Index', kind='line', title='Rental Index Trend (2005-2023)', grid=True)

plt.show()

In [ ]:
merged_df['Date'].unique()

In [ ]:
df_2023 = merged_df[merged_df['Date'] == '2023-12-01']
grouped_df = df_2023.groupby('Geography').agg(Rental_Index_Dec2023=('Rental_Index', 'mean')).reset_index()


In [ ]:
grouped_df.sort_values(by='Rental_Index_Dec2023', inplace=True)

In [ ]:
grouped_df.plot(x='Geography', y='Rental_Index_Dec2023', kind='bar', title='UK Regional Rental Index Comparison (Dec 2023)', grid=True, figsize=(10,7))

In [ ]:
london_df = merged_df[merged_df['Geography'] == 'London']
east_midlands_df = merged_df[merged_df['Geography'] == 'East Midlands']

In [ ]:
plt.plot(london_df['Date'], london_df['Rental_Index'], label='London', color='blue')
plt.plot(east_midlands_df['Date'], east_midlands_df['Rental_Index'], label='East Midlands', color='red')
plt.title('London vs East Midlands Rental Trends')
plt.title('London vs East Midlands Rental Trends')
plt.xlabel('Year')
plt.ylabel('Rental Index')

plt.legend()

plt.grid(True)
plt.show()

In [ ]:
# For London
london_start = london_df[london_df['Date'] == '2005-01-01']['Rental_Index'].values[0]
london_end = london_df[london_df['Date'] == '2023-12-01']['Rental_Index'].values[0]
london_growth = (london_end - london_start) / london_start * 100

# For East Midlands
east_mid_start = east_midlands_df[east_midlands_df['Date'] == '2005-01-01']['Rental_Index'].values[0]
east_mid_end = east_midlands_df[east_midlands_df['Date'] == '2023-12-01']['Rental_Index'].values[0]
east_mid_growth = (east_mid_end - east_mid_start) / east_mid_start * 100

# Annualized growth
years = 2023 - 2005 + 1  # 19 years
london_annual = (1 + london_growth/100)**(1/years) - 1
east_mid_annual = (1 + east_mid_growth/100)**(1/years) - 1

print(london_start)
print(london_end)
print(london_growth)
print(london_annual)
print(east_mid_start)
print(east_mid_end)
print(east_mid_growth)
print(east_mid_annual)

In [88]:
corr_df = england_df[['CPIH_Inflation_All','Rental_Index']].reset_index(drop=True)

In [90]:
corr_df.corr()

,CPIH_Inflation_All,Rental_Index
CPIH_Inflation_All,1.000000,0.345928
Rental_Index,0.345928,1.000000
